<h1 style="text-align: center;">Tennis Analyzer ML Model V2</h1>
<h3 style="text-align: center;">Dan Warnick</h3>

<p>To start we will begin by selecting the data points we want to analyze with existing known results. For each data entry we will have two players each with the following data entries.</p>
<table style="font-size: .8em;">
    <tr>
        <th>Player Name</th>
    </tr>
</table>
<p>Along with match facts like Clay/Hard/Grass Court or Indoor/Outdoor. In the future may want to add weather and adjust certain parameters for more accuracy and more data points to train from, however for now this seems a good start.</p>

<h2>1.) Collect Data Efficiently</h2>

In [218]:
import os
import copy
import joblib
import math
from datetime import datetime
from math import ceil, floor

import django
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from asgiref.sync import sync_to_async  # type: ignore
from django.db import models
from django.http import HttpResponse
from django.template import loader
from lightgbm import LGBMClassifier
from optuna.integration import lightgbm as lgb_optuna
from scipy.special import erf, erfinv
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import lightgbm as lgb
from tqdm.notebook import tqdm


# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'breakpoint.settings')
# django.setup()

# from render.models import *

In [219]:
start = '20120101'
end = '20231231'
match_type = 'm'

In [220]:
start_date = datetime.strptime(start, '%Y%m%d').date()
end_date = datetime.strptime(end, '%Y%m%d').date()

# if match_type == 'm':   
#     typer = MensTennisMatch
#     insert_db = MensTennisMatchStats
# else:
#     typer = WomensTennisMatch
#     insert_db = WomensTennisMatchStats

# query = insert_db.objects.filter(
#         tourney_date__range=(start_date, end_date)
#     ).order_by('tourney_date')
    
# games = await sync_to_async(list)(query.all().values())

df = pd.read_csv('testcsvs/glickoalltest.csv')
df['tourney_date'] = pd.to_datetime(df['tourney_date']).dt.date
df = df[(df['tourney_date'] >= start_date) & (df['tourney_date'] <= end_date)]
df = df[(df['a_glicko_rd'] <= 150) & (df['b_glicko_rd'] <= 150)]


df = df.drop(['tourney_id', 'tourney_name', 'match_num', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug', 'sets', 'games', 'tiebreaks'], axis=1)

# df = pd.DataFrame(games).drop(['tourney_id', 'tourney_name', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug','a_win_percent', 'a_serve_rating', 'a_return_rating', 'a_pressure_rating', 'a_avg_vs_elo', 'a_matches_played', 'b_win_percent', 'b_serve_rating', 'b_return_rating', 'b_pressure_rating', 'b_avg_vs_elo', 'b_matches_played', 'A_Odds', 'b_odds'], axis=1)

one_hot_encoder = OneHotEncoder(sparse_output=False)

# One-Hot Encode the 'category_text' column
category_encoded = one_hot_encoder.fit_transform(df[['surface']])

# Convert to DataFrame
category_encoded_df = pd.DataFrame(category_encoded, columns=one_hot_encoder.get_feature_names_out(['surface']))

# Concatenate the one-hot encoded columns back to the original DataFrame
df = pd.concat([df, category_encoded_df], axis=1)

# Drop the original 'category_text' column
df.drop('surface', axis=1, inplace=True)
print(len(df))

36157


In [221]:
df.head()

,best_of,a_player_rank,b_player_rank,glicko_rating_diff_high,glicko_rating_diff_low,a_glicko_rating,b_glicko_rating,a_glicko_rd,b_glicko_rd,point_glicko_rating_diff_high,...,surface_return_second_won_glicko_rating_diff_high,surface_return_second_won_glicko_rating_diff_low,a_surface_return_second_won_glicko_rating,b_surface_second_won_glicko_rating,a_odds,b_odds,a_b_win,surface_Clay,surface_Grass,surface_Hard
5373,3.0,74.0,15.0,-311.617231,-40.704658,1663.254111,1839.415055,69.063534,66.392752,-145.321147,...,-141.831083,110.643651,1514.815163,1530.408879,3.59,1.28,0.0,0.0,0.0,1.0
5375,3.0,65.0,46.0,-147.093176,121.020149,1691.581925,1704.618439,67.784328,66.272335,-128.899533,...,-124.671282,124.710330,1524.284813,1524.265289,NaN,NaN,1.0,0.0,0.0,1.0
5377,3.0,95.0,89.0,-181.835201,123.629608,1570.250330,1599.353126,82.839194,69.893210,-124.286662,...,-126.885213,130.276738,1510.001918,1508.306155,1.59,2.29,1.0,0.0,0.0,1.0
5378,3.0,48.0,83.0,-45.128121,225.802369,1730.873087,1640.535963,65.783232,69.682012,-126.628591,...,-124.858464,126.689605,1508.449284,1507.533713,2.40,1.54,0.0,0.0,0.0,1.0
5380,3.0,70.0,22.0,-452.184818,-159.378373,1545.426123,1851.207719,79.493396,66.909826,-168.130661,...,-142.624208,109.143893,1486.596303,1503.336460,4.44,1.19,0.0,0.0,0.0,1.0


In [222]:
# Create a scatter plot
# df = df[~(df == 1500).any(axis=1)]
# plt.figure(figsize=(5, 5))

# # Plot a_elo_rating vs b_elo_rating for a_b_win == 1
# x1 = df[df['a_b_win'] == 1]['a_recent_elo_rating']
# y1 = df[df['a_b_win'] == 1]['b_recent_elo_rating']
# plt.scatter(x1, y1, color='blue', label='Favorite Wins', s=.5, alpha=0.5)

# # Plot b_elo_rating vs a_elo_rating for a_b_win == 0
# x2 = df[df['a_b_win'] == 0]['b_recent_elo_rating']
# y2 = df[df['a_b_win'] == 0]['a_recent_elo_rating']
# plt.scatter(x2, y2, color='orange', label='Upset', s=.5, alpha=0.5)

# # Combine data for a single trendline
# combined_x = np.concatenate([x1, x2])
# combined_y = np.concatenate([y1, y2])

# # Fit a polynomial of degree 2 to the combined data
# print(combined_x)
# coefficients = np.polyfit(combined_x, combined_y, 3)
# polynomial = np.poly1d(coefficients)
# trendline_x = np.linspace(combined_x.min(), combined_x.max(), 100)
# trendline_y = polynomial(trendline_x)
# plt.plot(trendline_x, trendline_y, color='green', linewidth=1, label='Quadratic Trendline')

# # Setting the limits for x and y axis
# plt.xlim(1100, 2300)
# plt.ylim(1100, 2300)

# # Adding labels and title
# plt.xlabel('ELO Rating Winner')
# plt.ylabel('ELO Rating Loser')
# plt.title('Scatter Plot of ELO Ratings based on Win/Loss')
# plt.grid(True)
# plt.legend()

# # Show the plot
# plt.show()

In [223]:
# plt.figure(figsize=(8, 5))

# df['elo_diff'] = df['a_recent_elo_rating']

# # Create bins for ELO difference in intervals of 10
# bins = np.arange(df['elo_diff'].min(), df['elo_diff'].max() + 10, 10)
# labels = (bins[:-1] + bins[1:]) / 2
# df['elo_diff_bin'] = pd.cut(df['elo_diff'], bins=bins, labels=labels, include_lowest=True)

# # Calculate average win rate at each ELO difference bin
# average_win_rate = df.groupby('elo_diff_bin')['a_b_win'].mean().reset_index()
# average_win_rate.columns = ['elo_diff_bin', 'avg_win_rate']

# # Convert the bin labels to numeric values
# average_win_rate['elo_diff_bin'] = average_win_rate['elo_diff_bin'].astype(float)

# average_win_rate = average_win_rate[~np.isnan(average_win_rate).any(axis=1)]

# # Create the scatter plot
# plt.scatter(average_win_rate['elo_diff_bin'], average_win_rate['avg_win_rate'], color='blue', label='Average Win Rate', s=10, alpha=0.5)

# # Fit a polynomial of degree 3 to the average win rate data
# coefficients = np.polyfit(average_win_rate['elo_diff_bin'], average_win_rate['avg_win_rate'], 5)
# polynomial = np.poly1d(coefficients)
# trendline_x = np.linspace(-600, 600, 100)
# trendline_y = polynomial(trendline_x)
# plt.plot(trendline_x, trendline_y, color='green', linewidth=1, label='Cubic Trendline')

# # Setting the limits for x and y axis
# plt.xlim(-600, 600)
# plt.ylim(0, 1)

# # Adding labels and title
# plt.xlabel('ELO Rating Difference (Winner - Loser)')
# plt.ylabel('Average Win Rate')
# plt.title('Scatter Plot of ELO Rating Difference vs. Win Rate')
# plt.grid(True)
# plt.legend()

# # Show the plot
# plt.show()

In [224]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [225]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata, norm

def rank_gauss_transform(series: pd.Series) -> pd.Series:
    """Applies the Rank Gauss transform to a pandas Series."""
    # Rank the data
    ranked = rankdata(series, method='average')
    # Scale the ranks to be between -1 and 1
    scaled_ranks = (ranked - 0.5) / len(series)
    # Apply the inverse normal (gaussian) CDF
    transformed = norm.ppf(scaled_ranks)
    return pd.Series(transformed, index=series.index)


In [226]:
df = df.dropna()
df = df[~(df == .5).any(axis=1)]
df = df[~(df == -20).any(axis=1)]

odds_df = df[['a_odds', 'b_odds']].copy()
odds_df['index'] = df.index

df = df.drop(columns=['a_odds', 'b_odds'])

y = df['a_b_win'].values
X = df.drop('a_b_win', axis=1).values

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, odds_df['index'].values, test_size=0.2, random_state=42
)

feature_names = df.drop('a_b_win', axis=1).columns.tolist()
X_train_df = pd.DataFrame(X_train, columns=feature_names)
X_test_df = pd.DataFrame(X_test, columns=feature_names)

# Apply Rank Gauss Transformation to all features
X_train_transformed = X_train_df.apply(rank_gauss_transform, axis=0)
X_test_transformed = X_test_df.apply(rank_gauss_transform, axis=0)

# Convert back to numpy arrays for modeling
X_train_transformed = X_train_transformed.values
X_test_transformed = X_test_transformed.values

# LightGBM Model Training
train_data = lgb.Dataset(X_train_transformed, label=y_train)
test_data = lgb.Dataset(X_test_transformed, label=y_test, reference=train_data)

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

lgbm_model = lgb.train(params, train_data, valid_sets=[train_data, test_data], num_boost_round=1000, callbacks=[lgb.early_stopping(stopping_rounds=50)])

# LightGBM Predictions and Evaluation
y_pred_lgbm = lgbm_model.predict(X_test_transformed)
y_pred_lgbm_binary = [1 if prob > 0.5 else 0 for prob in y_pred_lgbm]

accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm_binary)
roc_auc_lgbm = roc_auc_score(y_test, y_pred_lgbm)

print(f'LightGBM Accuracy: {accuracy_lgbm}')
print(f'LightGBM ROC AUC: {roc_auc_lgbm}')

# PyTorch Model Definition
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Convert transformed data to tensors for PyTorch
# Convert transformed data to tensors for PyTorch
X_train_tensor = torch.tensor(X_train_transformed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_transformed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# DataLoader for PyTorch
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize the PyTorch model, loss function, and optimizer
input_dim = X_train_transformed.shape[1]
model = BinaryClassificationModel(input_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the PyTorch Model
num_epochs = 50
model.train()

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# PyTorch Predictions and Evaluation
model.eval()
with torch.no_grad():
    y_pred_pytorch = model(X_test_tensor).squeeze()
    y_pred_pytorch_binary = (y_pred_pytorch > 0.5).float()

accuracy_pytorch = accuracy_score(y_test_tensor, y_pred_pytorch_binary)
roc_auc_pytorch = roc_auc_score(y_test_tensor, y_pred_pytorch)

print(f'PyTorch Accuracy: {accuracy_pytorch}')
print(f'PyTorch ROC AUC: {roc_auc_pytorch}')

# Comparison of Results
print(f'LightGBM Accuracy: {accuracy_lgbm}, ROC AUC: {roc_auc_lgbm}')
print(f'PyTorch Accuracy: {accuracy_pytorch}, ROC AUC: {roc_auc_pytorch}')


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[108]	training's binary_logloss: 0.518712	valid_1's binary_logloss: 0.592064
LightGBM Accuracy: 0.6805396096440872
LightGBM ROC AUC: 0.749610595109569
Epoch 10/50, Loss: 0.5437866449356079
Epoch 20/50, Loss: 0.4960002899169922
Epoch 30/50, Loss: 0.3898712396621704
Epoch 40/50, Loss: 0.3576592803001404
Epoch 50/50, Loss: 0.4251019060611725
PyTorch Accuracy: 0.6093570608495982
PyTorch ROC AUC: 0.637091784174915
LightGBM Accuracy: 0.6805396096440872, ROC AUC: 0.749610595109569
PyTorch Accuracy: 0.6093570608495982, ROC AUC: 0.637091784174915


In [227]:
# Display the DataFrame
# results_df.head()
print(results_df)

NameError: name 'results_df' is not defined

In [ ]:
# Kelly Criterion
def kelly_criterion(vegas_odds, calculated_probability):
    # Calculate the Kelly fraction
    kelly_fraction = (vegas_odds * calculated_probability - (1 - calculated_probability)) / vegas_odds
    
    # Ensure that the fraction is not negative
    kelly_fraction = max(0, kelly_fraction)
    
    return kelly_fraction

In [ ]:
better = 0
total_won = 0
diff_fav = 0
bet_correct = 0

upset_predict = 0
upset_correct = 0
upset_won = 0

model_correct = 0
vegas_correct = 0
vegas_total = 0

wrong = 0
comparison_df = results_df.dropna()
length = len(comparison_df)

confidence_pct = .5
confidence_top_pct = 1

UNIT = 10

for i, row in comparison_df.iterrows():

    if confidence_top_pct > row['Predicted'] > confidence_pct :#and row['Predicted'] > 1/row['A_Odds']:
        better += 1
        if(row['Actual'] == 1):
            bet_correct += 1
            total_won += (row['A_Odds']-1) * (kelly_criterion(row['A_Odds'],row['Predicted']) * UNIT)
        else:
            wrong += 1
            total_won -= (kelly_criterion(row['A_Odds'],row['Predicted']) * UNIT)

    if 1-confidence_top_pct < row['Predicted'] < 1-confidence_pct :#and 1-row['Predicted'] > 1/row['B_Odds']:
        better += 1
        if(row['Actual'] == 0):
            bet_correct += 1
            total_won += (row['B_Odds']-1) * (kelly_criterion(row['B_Odds'],row['Predicted']) * UNIT)
        else:
            wrong += 1
            total_won -= (kelly_criterion(row['B_Odds'],row['Predicted']) * UNIT)

    if round(row['Predicted']) != round(1/row['A_Odds']):
        upset_predict += 1
        if round(row['Predicted']) == round(row['Actual']):
            upset_correct += 1
            if(row['Actual'] == 1):
                upset_won += (row['A_Odds']-1) * UNIT
            else:
                upset_won += (row['B_Odds']-1) * UNIT
        else:
            upset_won -= UNIT

    if round(row['Predicted']) != round(1/row['A_Odds']):
        diff_fav += 1

    if row['Actual']==1 and row['A_Odds'] < row['B_Odds']:
        # print(f"A odds : {row['a_odds']} B odds : {row['b_odds']}")
        vegas_correct += 1

    if row['Actual']==0 and row['A_Odds'] > row['B_Odds']:
        vegas_correct += 1

    if round(row['Predicted']) == round(row['Actual']):
        model_correct += 1


print(f"Total won on ${UNIT} bets: {total_won:.2f} on a total # bets: {better} from a total of {length} games")
print(f"Amount of differing favorites {diff_fav/length}")
print(f"Amount of upset correct {upset_correct/upset_predict} won ${upset_won} on {upset_predict} bets")
print(f"Amount of incorrect bets : {wrong/better}")
print(f"Correct Bets: {bet_correct/better}")
print(f"Model % Correct : {model_correct/length} Vegas Correct % : {vegas_correct/length}")


Total won on $10 bets: -305.43 on a total # bets: 3484 from a total of 3484 games
Amount of differing favorites 0.13662456946039037
Amount of upset correct 0.4264705882352941 won $-212.89999999999992 on 476 bets
Amount of incorrect bets : 0.3145809414466131
Correct Bets: 0.6854190585533869
Model % Correct : 0.6854190585533869 Vegas Correct % : 0.7049368541905855
